# 🧠 Face Recognition & Text Extraction com OpenAI API

In [ ]:
import openai
import os
from PIL import Image, ImageDraw
import base64
import io
from dotenv import load_dotenv

# Carregar API Key
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# Função para converter imagem em base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# Função para exibir e salvar imagem recortada com base nas coordenadas
def crop_faces(image_path, face_coords_list):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    cropped_faces = []
    for i, coords in enumerate(face_coords_list):
        left, top, right, bottom = coords
        draw.rectangle([left, top, right, bottom], outline="red", width=3)
        cropped = image.crop((left, top, right, bottom))
        cropped_faces.append(cropped)
        cropped.save(f"face_{i+1}.jpg")

    image.save("image_with_faces.jpg")
    return cropped_faces

In [ ]:
# Caminho para a imagem de teste (substitua pelo seu arquivo)
image_path = "sua_imagem.jpg"
base64_image = image_to_base64(image_path)

In [ ]:
# Prompt para o GPT-4 Vision
prompt = (
    "Analise a imagem, identifique todos os rostos visíveis, retorne as coordenadas de cada rosto no formato: "
    "[[left, top, right, bottom], ...]. Em seguida, extraia todos os textos visíveis e liste-os separadamente."
)

In [ ]:
# Chamada à OpenAI API
response = openai.ChatCompletion.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }
    ],
    max_tokens=1200
)

# Exibir resposta
content = response["choices"][0]["message"]["content"]
print("Resposta do modelo:")
print(content)

In [ ]:
# Exemplo fictício de coordenadas (substitua pelos reais extraídos do content)
example_coords = [[50, 30, 150, 130]]
# Recortar rostos
# crop_faces(image_path, example_coords)